In [1]:
import os 
from dotenv import load_dotenv

os.chdir("..")
load_dotenv()


True

In [2]:
import random
import string
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta
from typing import Dict, Optional
from langchain.tools import BaseTool

class OTPManager:
    def __init__(self, expiry_minutes: int = 5):
        self.otps: Dict[str, Dict] = {}  # Store email -> {otp, expiry} mapping
        self.expiry_minutes = expiry_minutes
    
    def generate_otp(self, length: int = 6) -> str:
        """Generate a random OTP of specified length"""
        return ''.join(random.choices(string.digits, k=length))
    
    def create_otp(self, email: str) -> str:
        """Create and store OTP for an email"""
        otp = self.generate_otp()
        expiry = datetime.now() + timedelta(minutes=self.expiry_minutes)
        self.otps[email] = {"otp": otp, "expiry": expiry}
        return otp
    
    def verify_otp(self, email: str, otp: str) -> bool:
        """Verify if OTP is valid for the given email"""
        if email not in self.otps:
            return False
        
        stored_data = self.otps[email]
        if datetime.now() > stored_data["expiry"]:
            del self.otps[email]
            return False
            
        if stored_data["otp"] == otp:
            del self.otps[email]  # Remove OTP after successful verification
            return True
        return False

class EmailSender:
    def __init__(self, smtp_server: str, smtp_port: int, username: str, password: str):
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.username = username
        self.password = password
    
    def send_otp_email(self, to_email: str, otp: str) -> bool:
        """Send OTP via email"""
        try:
            msg = MIMEMultipart()
            msg['From'] = self.username
            msg['To'] = to_email
            msg['Subject'] = "Your OTP for Verification"
            
            body = f"Your OTP is: {otp}\nValid for 5 minutes."
            msg.attach(MIMEText(body, 'plain'))
            
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(self.username, self.password)
                server.send_message(msg)
            return True
        except Exception as e:
            print(f"Error sending email: {e}")
            return False



In [3]:
sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_APP_PASSWORD")

if not sender_email or not sender_password:
    raise ValueError("Email credentials not found in environment variables")

# SMTP server configuration
smtp_server = "smtp.gmail.com"
smtp_port = 587

otp_manager = OTPManager()
email_sender = EmailSender(smtp_server, smtp_port, sender_email, sender_password)

In [12]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field




# Note: It's important that every field has type hints. BaseTool is a
# Pydantic class and not having type hints can lead to unexpected behavior.

class SendOTPInput(BaseModel):
    email: str = Field(description="Email address to send OTP to")
    
    
@tool(name_or_callable="send_otp", args_schema=SendOTPInput)
def send_otp_tool(email:str):
    "Sends an OTP to the specified email address for verification"
    
    
    

    otp = otp_manager.create_otp(email)
    if email_sender.send_otp_email(email, otp):
        return f"OTP sent successfully to {email}"
    return f"Failed to send OTP to {email}"



class VerifyOTPInput(BaseModel):
    email: str = Field(description="Email address to verify OTP for")
    otp: str = Field(description="OTP to verify")

@tool(name_or_callable="verify_otp", args_schema=VerifyOTPInput)
def verify_otp_tool(email:str, otp:str):
    "Verifies the OTP provided by the user"
    
    
    if otp_manager.verify_otp(email, otp):
        return "OTP verified successfully"
    return "Invalid or expired OTP"






In [10]:
send_otp_tool.invoke({"email":"hrisikesh.neogi@gmail.com"})

'OTP sent successfully to hrisikesh.neogi@gmail.com'

In [13]:
verify_otp_tool.invoke(
    {"email":"hrisikesh.neogi@gmail.com",
     "otp":"664158"}
)

'OTP verified successfully'

ImportError: cannot import name 'format_phone_number' from partially initialized module 'src.tools' (most likely due to a circular import) (d:\real-estate-demo\src\tools\__init__.py)

: 